# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [21]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test` and then get `X_train`, `y_train`, `X_valid`, `y_valid` from the previous `X_train`, `y_train`. Use the additional parameter `stratify`.

In [4]:
df = pd.read_csv('../data/dayofweek.csv')

X = df.drop('dayofweek', axis=1)
y = df['dayofweek']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=21, stratify=y_train)

## 2. Individual classifiers

1. Train SVM, decision tree and random forest again with the best parameters that you got from the 01 exercise with `random_state=21` for all of them.
2. Evaluate `accuracy`, `precision`, and `recall` for them on the validation set.
3. The result of each cell of the section should look like this:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [6]:
def print_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print(f'accuracy is {accuracy:.5f}')
    print(f'precision is {precision:.5f}')
    print(f'recall is {recall:.5f}')

In [7]:
svm = SVC(C=10, class_weight='balanced', gamma='scale', kernel='rbf', random_state=21, probability=True)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_valid)
print_metrics(y_valid, y_pred)

accuracy is 0.86296
precision is 0.86755
recall is 0.86296


In [8]:
dt = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=23, random_state=21)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_valid)
print_metrics(y_valid, y_pred)

accuracy is 0.89259
precision is 0.89462
recall is 0.89259


In [9]:
rf = RandomForestClassifier(n_estimators=50, max_depth=39, class_weight='balanced', criterion='gini', random_state=21)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_valid)
print_metrics(y_valid, y_pred)

accuracy is 0.89259
precision is 0.89458
recall is 0.89259


## 3. Voting classifiers

1. Using `VotingClassifier` and the three models that you have just trained, calculate the `accuracy`, `precision`, and `recall` on the validation set.
2. Play with the other parameteres.
3. Calculate the `accuracy`, `precision` and `recall` on the test set for the model with the best weights in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).

In [13]:
voting = VotingClassifier(estimators=[
    ('svm', svm),
    ('dt', dt),
    ('rf', rf)
], voting='soft')

voting.fit(X_train, y_train)
y_pred = voting.predict(X_valid)

print_metrics(y_valid, y_pred)

accuracy is 0.90000
precision is 0.90145
recall is 0.90000


In [14]:
y_pred = voting.predict(X_test)
print_metrics(y_test, y_pred)

accuracy is 0.89645
precision is 0.89774
recall is 0.89645


## 4. Bagging classifiers

1. Using `BaggingClassifier` and `SVM` with the best parameters create an ensemble, try different values of the `n_estimators`, use `random_state=21`.
2. Play with the other parameters.
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision)

In [19]:
bagging_svm = BaggingClassifier(estimator=svm, random_state=21)

params = {
    'n_estimators': [10, 20, 50, 100],
    'max_samples': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
}

grid_search = GridSearchCV(estimator=bagging_svm, param_grid=params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_bagging_svm = grid_search.best_estimator_

y_pred = best_bagging_svm.predict(X_valid)
print_metrics(y_valid, y_pred)

accuracy is 0.88757
precision is 0.88904
recall is 0.88757


In [29]:
y_pred = best_bagging_svm.predict(X_test)
print_metrics(y_test, y_pred)

accuracy is 0.88757
precision is 0.88904
recall is 0.88757


## 5. Stacking classifiers

1. To achieve reproducibility in this case you will have to create an object of cross-validation generator: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, where `n` you will try to optimize (the details are below).
2. Using `StackingClassifier` and the three models that you have recently trained, calculate the `accuracy`, `precision` and `recall` on the validation set, try different values of `n_splits` `[2, 3, 4, 5, 6, 7]` in the cross-validation generator and parameter `passthrough` in the classifier itself,
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision). Use `final_estimator=LogisticRegression(solver='liblinear')`.

In [30]:
estimators = [
    ('svm', svm),
    ('dt', dt),
    ('rf', rf)
]

stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(solver='liblinear'))

params = {
    'cv': [StratifiedKFold(n_splits=n, shuffle=True, random_state=21) for n in [2, 3, 4, 5, 6, 7]],
    'passthrough': [True, False]
}
grid_search = GridSearchCV(estimator=stacking_clf, param_grid=params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_stacking_clf = grid_search.best_estimator_

y_pred = best_stacking_clf.predict(X_valid)
print_metrics(y_valid, y_pred)

accuracy is 0.88889
precision is 0.89071
recall is 0.88889


In [31]:
y_pred = best_stacking_clf.predict(X_test)
print_metrics(y_test, y_pred)

accuracy is 0.89941
precision is 0.90210
recall is 0.89941


## 6. Predictions

1. Choose the best model in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).
2. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which labname and for which users.
3. Save the model.

In [32]:
best_model = best_stacking_clf
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Final Accuracy:", accuracy)

Final Accuracy: 0.8994082840236687


In [33]:
result = pd.DataFrame({'actual': y_test, 'predicted': y_pred})

errors = result[result['actual'] != result['predicted']]['actual'].value_counts(normalize=True) * 100

errors_df = pd.DataFrame({'weekday': errors.index, 'error_percentage': errors.values})
errors_df['weekday'] = errors_df['weekday'].replace({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})

errors_df

,weekday,error_percentage
0,Saturday,26.470588
1,Sunday,20.588235
2,Monday,20.588235
3,Tuesday,8.823529
4,Friday,8.823529
5,Wednesday,8.823529
6,Thursday,5.882353


In [34]:
from joblib import dump

filepath = '../data/stacking_model.joblib'
dump(best_model, filepath)

['../data/stacking_model.joblib']